# Basic Model of Natural Selection
This Python notebook is the interactive version of the ["Basic Model of Natural Selection" Walk in the Forest post](https://walkintheforest.com/content/post/Basic+Model+of+Natural+Selection).

## Table of Contents
1. [Module Imports](#module-imports)
2. [Defining the Fitness Functions and Variable](#def-fitness-variable)
    - [Plotting the Fitness Function](#plot-fit-functions)
3. [Building the Model](#build-model)
    - [Visualization Function](#visualization-func)
    - [Overall Model Function](#overall-model-func)
    - [No Selection Conditions](#no-select-conditions)
    - [Basic Selection](#basic-selection)
        - [Basic Selection (Medium-Complexity Environment)](#basic-selection-medium)
        - [Basic Selection (High-Complexity Environment)](#basic-selection-high)
    - [Improved Selection](#improved-selection)
        - [Plotting Averages of Multiple Runs](#improved-selection-averages)

<a id='module-imports'></a>
## Module Imports

In [23]:
import numpy as np
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

<a id='def-fitness-variable'></a>
## Defining the Fitness Functions and Variable
We first start by defining our fitness functions and range for our variable, `var1`.

In [24]:
### Define grid of values for var1 ###
var1 = np.linspace(0,4*math.pi, 1000)

### Define our three fitness functions ###

# Simple Horizontal Line
simple_func = [2 for x in var1] 

# Simple Periodic Function
def med_func(x):
    """ Sin function
    """
    return(2*np.sin(x) + 2)

# Complex Periodic Function
def high_func(x):
    """ Complex periodic function
    """
    return(np.sin(x) + 2*np.sin(1.2*x + 1)**2 + 1)

<a id='plot-fit-functions'></a>
### Plotting the Fitness Functions
Before we start building our model, let's quickly plot all of our fitness functions.

In [25]:
## Creating the overall figure
landscape_plots = make_subplots(rows=1, cols=3,
                                horizontal_spacing=0.08,
                                vertical_spacing=0.08,
                                subplot_titles=("Low", "Medium", "High"),
                                shared_yaxes=True)

## Adding each environment to the figure
landscape_plots.add_trace(go.Scatter(x=var1, y=simple_func, name="Low"), row=1, col=1)
landscape_plots.add_trace(go.Scatter(x=var1, y=med_func(var1), name="Medium"), row=1, col=2)
landscape_plots.add_trace(go.Scatter(x=var1, y=high_func(var1), name="High"), row=1,col=3)

## Clean up figure
landscape_plots.update_layout(title="Fitness Landscapes of Different Complexity",
                              title_x = 0.5,
                              template="plotly_dark",
                              autosize=True,
                              showlegend=False)

In [4]:
landscape_plots.write_html("figures/landscape_plots.html")

<a id='build-model'></a>
## Building the Model
While we will be building the model step-by-step and exploring different examples and levels of complexity, we can describe the model and its visualization as a set of six steps.

1. Initialize the starting variant
2. Use our generation algorithm to create the next generation
3. Evaluate the next generation using our fitness function
4. Determine the next generation
5. Repeat Steps 1-4 as many times as designated.
6. Visualize the model using a graphing library (Plotly)

We can further subdivide the model into two parts: data generation (Steps 1-5) and visualization (Step 6).

<a id='visualization-func'></a>
### Visualization Function
While unconvetional, it will be easiest to introduce the general visualization function before building our model. We will be exploring multiple examples and levels of complexity during data generation and visualizing each example will be key to understanding the process. To visualize our model, we will use the [Plotly](https://plotly.com/python/) library. This function will take in data from a single model run and overlay it on a static graph of the fitness function one generation at a time.

**Note: Understanding the animating code is not neccessary to understanding the model. It just provides a unique window into the process**

In [26]:
def make_plot(num_gens, var1, fit_func, gen_var1, gen_fitness, title):
    """ Create animations of single runs of the natural selection model

    Args:
        num_gens (int): Number of generations to run
        var1 (list): Grid of values for var1 for landscape plotting
        fit_func (function): Fitness function
        gen_var1 (list): List of var1 values for each generation
        gen_fitness (list): List of fitness values for each generation
        title (string): Title for the model to use for graphing
    
    Returns:
        fig (Plotly Figure): Final animated figure

    """
    
    ## Make initial subplot figure
    fig = make_subplots(rows=2, cols=1,
                        subplot_titles=("Fitness Landscape", "Fitness over Generations"))
    
    ## Calculate landscape
    fit_landscape = fit_func(var1)
    max_fit = max(fit_landscape)
    min_fit = min(fit_landscape)
    
    # Add holding traces for animation frames
    
    ## Fitness Landscape
    fig.add_trace(go.Scatter(x=var1,
                             y=fit_landscape), 
                  row=1, col=1)
    
    ## Newest Generation
    fig.add_trace(go.Scatter(x=[gen_var1[0]],
                             y=[gen_fitness[0]],
                             mode="markers",
                             marker=dict(size=15)),
                  row=1,col=1)
    
    ## Previous Generation
    fig.add_trace(go.Scatter(mode="markers"),
                 row=1,col=1)
    
    ## Fitness for each Generation (second subplot)
    fig.add_trace(go.Scatter(x=[0],
                             y=[gen_fitness[0]],
                             mode="markers+lines"),
                  row=2, col=1) 
    
    # Update subplot axies
    fig.update_xaxes(title= "var1", row=1,col=1)
    fig.update_yaxes(range=[min_fit-.2, max_fit+.2],title="Fitness", row=1,col=1)
    fig.update_xaxes(range=[0,num_gens],title="Generation", row=2,col=1)
    fig.update_yaxes(range=[min_fit-.2, max_fit+.2], title="Fitness", row=2,col=1)
    
    # Create animation frames from data
    frames = [dict(
                  name = k,
                  data = [go.Scatter(x=[gen_var1[k]], y=[gen_fitness[k]]),
                          go.Scatter(x=gen_var1[:(k+1)], y=gen_fitness[:(k+1)]),
                          go.Scatter(x=list(range(k+1)),y=gen_fitness[:(k+1)])
                          ],
                  traces = [1,2,3]
                  ) for k in range(num_gens)]
    
    # Create Play and Pause Buttons
    updatemenus = [
        {
            "buttons" : [
                {
                    "label" : "Play",
                    "method" : "animate",
                    "args" : [None, {"fromcurrent": True}]
                },
                {
                    "label" : "Pause",
                    "method": "animate",
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}]
                }
            ],
            "type" : "buttons",
            "showactive": False,
            "direction" : "right",
            "xanchor" : "left",
            "yanchor" : "top",
            "x" : 0,
            "y" : 0,
            "pad" : {"r":10, "t":30}            
        }
    ]
    
    # Final figure creation and updates
    fig.update(frames=frames)
    fig.update_layout(updatemenus=updatemenus, showlegend=False,
                      title=title, autosize=True,
                      template="plotly_dark",
                      title_x = 0.5)
    return(fig)

<a id='overall-model-func'></a>
### Overall Model Function
Because we will be running the same general model with different starting data, environment functions, and selection functions, let's define a general-purpose model-running function that focuses only on *data generation*. 

In [27]:
def nat_sel_model(num_gens, std_dev, var1_start, state, fit_func, sel_func):
    """ Overall function to generate data for a single run

    Args:
        num_gens (int): Number of generations to run
        std_dev (float): Value used to generate next var1
        var1_start (float): Initial generation's var1 value
        state (RandomState): Numpy seed for random number generation
        fit_func (Function): Fitness function for evaluating fitness
        sel_func (Function): Selection algorithm to determine variant

    Returns:
        var1_list (list): List of var1 values for each generation
        fit_list (list): List of fitness values for each generation

    """
    
    # Initialize our model
    var1_list, fit_list = init_gens(num_gens, var1_start, fit_func)
    
    # Run the generation and selection algorithms for a set number of generations
    sel_func(num_gens, var1_list, fit_list, fit_func, std_dev, state)
    
    return(var1_list, fit_list)

<a id='no-select-conditions'></a>
### No Selection Conditions
Before we start creating and selecting new variants, we first need a way of storing the data for visualization and creating the initial generation. This represents Step 1 from our model outline.

In [28]:
def init_gens(num_gens, var1_start, fit_func):
    """ Initialize lists and initial conditions for var1 and fitness

    Args:
        num_gens (int): Number of generations to run
        var1_start (float): Initial generation's var1 value
        fit_func (Function): Fitness function for evaluating fitness

    Returns:
        var1_list (list): List with only first var1 value filled
        fit_list (list): List with only first fitness value filled

    """
    
    var1_list = np.zeros(num_gens)
    fit_list = np.zeros(num_gens)
    var1_list[0] = var1_start
    fit_list[0] = fit_func(var1_list[0])
    return(var1_list, fit_list)

Now that we have a function that can prepare our data storage and the first generation, let's create an additional function to help generate the next potential variant. This represents Step 2 from our model outline.

In [29]:
def repro_alg(prev_var1, fit_func, std_dev, state):
    """Generate a new variant and associated fitness
    
    Args:
        prev_var1 (float): Previous value for var1
        fit_func (function): Fitness function to evaluate fitness using var1
        std_dev (float): Value used to generate next var1
        state (RandomState):
    
    Returns:
        new_var1 (float): New value for var1
        new_fitness (float): New fitness value associated with new_var1
    """
    
    new_var1 = state.normal(prev_var1, std_dev)
    new_fit = fit_func(new_var1)
    return(new_var1, new_fit)

Finally, we define the method we will use to determine the next generation. For this initial example, we won't utilize any selection conditions—we will accept *any new variant*, regardless of whether it has a higher or lower fitness than the previous variant. This represents Step 3 from our model outline.

In [30]:
def no_selection(num_gens, var1_list, fit_list, fit_func, std_dev, state):
    """ Generates data with no selection

    Args:
        num_gens (int): Number of generations to run
        var1_list (list): List with only first var1 value filled
        fit_list (list): List with only first fitness value filled
        fit_func (Function): Fitness function for evaluating fitness
        std_dev (float): Value used to generate next var1
        state (RandomState): Numpy seed for random number generation

    Returns:
        bool: True for success. False otherwise.

    """
    for i in range(1,num_gens):
        var1_list[i], fit_list[i] = repro_alg(var1_list[i-1], fit_func, std_dev, state)

Now that we have defined functions for all of the steps in the model, we can initiate a single run.

In [31]:
### Define the model conditions and RNG Seed
ex1_num_gens = 50
ex1_std_dev = 0.3
ex1_start = 4
ex1_state = np.random.RandomState(123)

# Generate the data to run the model
ex1_var1, ex1_fit = nat_sel_model(ex1_num_gens, ex1_std_dev,
                                  ex1_start, ex1_state, med_func, no_selection)

# Generate plot
no_sel_med_env_plot = make_plot(ex1_num_gens, var1, med_func, ex1_var1, ex1_fit,
                                title="No Selection (Medium Complexity)")
no_sel_med_env_plot.show()

In [22]:
no_sel_med_env_plot.write_html("figures/no_sel_med_env_plot.html")

<a id='basic-selection'></a>
### Basic Selection
Normally, we would expect to see a trend towards and stabilizing at a maximum. However, in our first implementation, we only see a random distribution of points near the starting value. To implement this change, we are going to add a new condition during data generation process. Instead of keeping any variant regardless of its fitness score, we will **keep a variant only if it improves on the current generation**. If it doesn't improve, then we'll keep the current generation and move on to another round.

In [32]:
def basic_selection(num_gens, var1_list, fit_list, fit_func, std_dev, state):
    """ Accepts every variant

    Args:
        num_gens (int): Number of generations to run
        var1_list (list): List with only first var1 value filled
        fit_list (list): List with only first fitness value filled
        fit_func (Function): Fitness function for evaluating fitness
        std_dev (float): Value used to generate next var1
        state (RandomState): Numpy seed for random number generation

    Returns:
        bool: True for success. False otherwise.

    """

    for i in range(1, num_gens):
        var1_list[i], fit_list[i] = repro_alg(var1_list[i-1], fit_func, std_dev, state)
    
        # Accept only if fitness increases
        if fit_list[i] < fit_list[i-1]:
            var1_list[i] = var1_list[i-1]
            fit_list[i] = fit_list[i-1]

<a id='basic-selection-medium'></a>
#### Basic Selection (Medium-Complexity Environment)
To start, let's implement this new process using the same medium-comeplexity environment used in the previous implementation.

In [33]:
### Define the model conditions and RNG Seed
ex2_num_gens = 30
ex2_std_dev = 0.3
ex2_start = 5
ex2_state = np.random.RandomState(123)

# Generate the data to run the model
ex2_var1, ex2_fit = nat_sel_model(ex2_num_gens, ex2_std_dev,
                                  ex2_start, ex2_state, med_func, basic_selection)

# Generate plot
basic_sel_med_env_plot = make_plot(ex2_num_gens, var1, med_func, ex2_var1, ex2_fit,
                                   title="Basic Selection (Medium Complexity)")
basic_sel_med_env_plot.show()

In [14]:
basic_sel_med_env_plot.write_html("figures/basic_sel_med_env_plot.html")

<a id='basic-selection-high'></a>
#### Basic Selection (High-Complexity Environment)
Now, let's apply this method to the higher complexity environment, the rightmost curve from Figure 1.

In [34]:
### Define the model conditions and RNG Seed
ex3_num_gens = 30
ex3_std_dev = 0.5
ex3_start = 4.8
ex3_state = np.random.RandomState(123)

# Generate the data to run the model
ex3_var1, ex3_fit = nat_sel_model(ex3_num_gens, ex3_std_dev,
                                  ex3_start, ex3_state, high_func, basic_selection)

# Generate Plot
basic_sel_high_env_plot = make_plot(ex3_num_gens, var1, high_func, ex3_var1, ex3_fit,
                                    title="Basic Selection (Medium Complexity)")
basic_sel_high_env_plot.show()

In [16]:
basic_sel_high_env_plot.write_html("figures/basic_sel_high_env_plot.html")

Now that we have a more complex landscape, you may see a potential problem in the current implementation: the trend gets stuck at *any peak* even if there are higher peaks around it. We call these smaller peaks "local maxima," since they represented a high point in a small region, but are not neccessarily the highest peak in the entire landscape. Our model will trend upwards to the nearest peak, but cannot jump across a valley because that would require a temporary drop in fitness.

<a id='improved-selection'></a>
### Improved Selection
Since our basic selection conditions didn't produce the behavior we are aiming for, we will add an additional step (a simplified Metropolis-Hastings algorithm) to accept variants with lower fitness with some probability defined by the magnitude of the loss in fitness.

In [35]:
def improved_selection(num_gens, var1_list, fit_list, fit_func, std_dev, state):
    """ Generates data with no selection

    Args:
        num_gens (int): Number of generations to run
        var1_list (list): List with only first var1 value filled
        fit_list (list): List with only first fitness value filled
        fit_func (Function): Fitness function for evaluating fitness
        std_dev (float): Value used to generate next var1
        state (RandomState): Numpy seed for random number generation

    Returns:
        bool: True for success. False otherwise.

    """

    for i in range(1,num_gens):
        new_var1, new_fitness = repro_alg(var1_list[i-1], fit_func, std_dev, state)
        
        # Calculate Change in Fitness
        delta_f = new_fitness - fit_list[i-1]
        
        # Run Selection
        if delta_f >= 0: # It improved
            var1_list[i] = new_var1
            fit_list[i] = new_fitness
        else:
            # Define threshold
            prob_scale = 0.5 # Rescale delta_f to increase/decrease probability
            threshold = np.exp(-abs(delta_f)/prob_scale)
            
            # Run Check
            if state.uniform(0,1) < threshold:
                var1_list[i] = new_var1
                fit_list[i] = new_fitness
            else:
                var1_list[i] = var1_list[i-1]
                fit_list[i] = fit_list[i-1]

In [36]:
### Initialize data storage
ex4_num_gens = 75
ex4_std_dev = 0.5
ex4_start = 4.9
ex4_state = np.random.RandomState(4) # Sets random seed for reproducibility

ex4_var1, ex4_fit = nat_sel_model(ex4_num_gens, ex4_std_dev,
                                  ex4_start, ex4_state, high_func, improved_selection)

improv_sel_high_env_plot = make_plot(ex4_num_gens, var1, high_func,
                                   ex4_var1, ex4_fit,
                                   title="Improved Selection (High Complexity)")
improv_sel_high_env_plot.show()

In [19]:
improv_sel_high_env_plot.write_html("figures/improved_sel_high_env_plot.html")

<a id='improved-selection-averages'></a>
#### Plotting Averages of Multiple Runs
In all of the previous examples, we have visualized a single run of our model for a finite number of generations. However, it's important to understand the average behavior of model across **multiple runs**. For any single run, the exact behavior may not converge at the highest peak, at least within the specified number of generations.

In [37]:
# Prepare the primary figure
average_fig = go.Figure()

# Setup general starting conditions
ex5_num_gens = 75
ex5_std_dev = 0.5
ex5_start = 4.9
ex5_state = np.random.RandomState(4)
num_runs = 100

# Setup storage for averages
avg_var1 = np.zeros(ex5_num_gens)
avg_fit = np.zeros(ex5_num_gens)

# Run the model multiple times
for i in range(num_runs):
    # Generate Model Data
    ex5_var1, ex5_fit = nat_sel_model(ex5_num_gens, ex5_std_dev,
                                      ex5_start, ex5_state, high_func, improved_selection)
    
    # Store to calculate averages for each generation                  
    avg_fit = np.add(avg_fit, ex5_fit)
    
    # Create trace for the run
    average_fig.add_trace(go.Scatter(x=list(range(ex5_num_gens)),
                                     y=ex5_fit, mode="lines",
                                     line={"color": 'rgba(200, 200, 200, 0.05)'},
                                     showlegend=False))

# Calculate Average
avg_fit = avg_fit/num_runs

# Plot average over individual runs
average_fig.add_trace(go.Scatter(x=list(range(ex5_num_gens)),y=avg_fit,
                                 mode="lines", name="Average"))
# Final layout changes
average_fig.update_xaxes(title="Generation")
average_fig.update_yaxes(title="Fitness")
average_fig.update_layout(template="plotly_dark",
                          title="100-Run Average with Improved Selection Model",
                          title_x = 0.5)
    

In [21]:
average_fig.write_html("figures/average-100-improved-selection.html")

The averaged plot illustrates that the model *does*, on average, trend towards improved variants over time. In addition, there are three clear convergent points (~2.5, ~3.1, and ~3.9) that correspond to three of the peaks in the landscape.